# Multicollinearity

### 可解釋性

很多情況下，模型的演算如果過於複雜，導致人類難以理解其決策與行動背後的原因，就會被視為黑盒子（black box），這類模型也被叫做黑盒模型

<img src="pics/black box model.png" alt="black box model" style="width: 500px">

可解釋的模型可以幫助我們輕鬆打開黑盒子，並且
1. 改善模型
2. 找出新現象，推論因果

架構複雜的模型往往具有較好的表現，但是可解釋性也較低

<img src="pics/interpretability.webp" alt="interpretability" style="width: 500px">

### 共線性

多重共線性（Multicollinearity）是指線性回歸模型中的解釋(自)變量之間由於存在精確相關關係或高度相關關係而使模型估計失真或難以估計準確

    假設今天有一條迴歸線為
<img src="pics/formular1.png" alt="formular1" style="width: 300px">

這時加入了另一就自變數X<sub>4</sub>，X<sub>4</sub>與X<sub>3</sub>高度相關，例如X<sub>4</sub> = 2*X<sub>3</sub>

此時公式變為：

<img src="pics/formular2.png" alt="formular2" style="width: 320px">

由於兩邊要找的是同樣的線性關係，在數學上兩者應該為等價關係

<img src="pics/formular3.png" alt="formular3" style="width: 400px">

後者的W<sub>5</sub>可以這樣表示

<img src="pics/formular4.png" alt="formular4" style="width: 130px">

p.s.注意此處W<sub>3</sub>與一開始的W<sub>3</sub>不等值

這個方程有無窮多的解，假設今天W<sub>5</sub>等於10，W<sub>3</sub>=2，W<sub>4</sub>=4是一個解，W<sub>3</sub>=8，W<sub>4</sub>=1也是一個解。

這會導致了模型係數估計的不穩定，也就很難找出各X和y之間的真實關係了。

### 多重共線性檢驗-變異數膨脹係數 Variance Inflation Factor (VIF)
計算每一個自變數和其他自變數的R<sup>2</sup>，便能得到VIF值，公式如下：

<img src="pics/VIF.png" alt="VIF" style="width: 130px">

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(8)

我們根據

    fuel consumed=1 × mph

這個事實來創建一個dataset，並且創造兩個完全共線性的feature。

mph : 英里/小時

kmph : 公里/小時，從mph轉換而來(1.6倍)，兩者為完全共線性

n_unicorn : 某處獨角獸的數量

Idontknow : 我也不知道


In [3]:
# 每英里/小時
df               = pd.DataFrame(np.arange(0, 100, 0.5), columns=["mph"])
# 每公里/小時
df['kmph']       = df['mph'] * 1.609344
# 幾隻獨角獸
df['n_unicorns'] = np.random.randint(low= 1, high=  4, size=df.shape[0])
#我也不知道
df['Idontknow']        = np.random.uniform(low= 0, high= 50, size=df.shape[0])
# we shall also add a little Gaussian noise to the target
df['fuel']       = df['mph'] * (1 + np.random.normal(0,0.02, size=df.shape[0]))

df.head()

,mph,kmph,n_unicorns,Idontknow,fuel
0,0.0,0.000000,1,18.378527,0.000000
1,0.5,0.804672,2,12.032513,0.490364
2,1.0,1.609344,2,2.411589,0.997131
3,1.5,2.414016,2,8.758339,1.513638
4,2.0,3.218688,3,27.838772,1.961464


In [8]:
# loc v.s iloc
X = df.iloc[:,[0,1,2,3]]
y = df.iloc[:,[4]]

In [16]:
from sklearn import linear_model

regression = linear_model.LinearRegression()
# ground truth: X=mph, y=fuel
regression.fit(df.loc[:, ["mph"]] , df.loc[:, ["fuel"]])
print("Ground truth regression coefficient:",regression.coef_)
print()

Ground truth regression coefficient: [[1.00019424]]



當我們單獨對燃料和mph做線性回歸時，我們得到的ground truth coefficient為 1

In [10]:
# full regression, this time using statsmodels
import statsmodels.api as sm

model   = sm.OLS(y,X)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                   fuel   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.612e+05
Date:                Mon, 04 Sep 2023   Prob (F-statistic):                        0.00
Time:                        00:03:21   Log-Likelihood:                         -313.99
No. Observations:                 200   AIC:                                      634.0
Df Residuals:                     197   BIC:                                      643.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

然後完整的計算出回歸的係數卻發現，迴歸線為

**fuel = 0.2787 * mph + 0.4486 * kmph**

我們無法說他是錯誤的，但是卻難以解釋，另外也可看出獨角獸還不知道幹嘛的特徵對於fuel幾乎沒有影響。

In [21]:
# variance_inflation_factor的原始碼
from statsmodels.regression.linear_model import OLS

def variance_inflation_factor(exog, exog_idx):
    # 自變數(x)的數量
    k_vars = exog.shape[1]
    # 轉成array
    exog = np.asarray(exog)
    # 我們要計算VIF的目標自變數
    x_i = exog[:, exog_idx]
    # 去除exog_idx欄位，避免與自己做回歸
    mask = np.arange(k_vars) != exog_idx
    x_noti = exog[:, mask]
    # 獲取最小平方的R平方值
    r_squared_i = OLS(x_i, x_noti).fit().rsquared
    # print(r_squared_i)
    # VIF值越大，表示共線性越嚴重
    vif = 1. / (1. - r_squared_i)

    return vif

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

VIF             = pd.DataFrame()
VIF['feature']  = X.columns
VIF['VIF']      = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

VIF

/Users/bananaiselite/anaconda3/envs/test/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,feature,VIF
0,mph,inf
1,kmph,inf
2,n_unicorns,3.426534
3,Idontknow,3.031613


通常VIF < 5我們就認為有弱共線性，5 < VIF < 10就為強共線性，mph和kmph毫不意外的有超高共線性，那我們就把kmph丟掉看看。

In [24]:
X = X.drop(['kmph'], axis=1)

VIF            = pd.DataFrame()
VIF['feature'] = X.columns
VIF['VIF']     = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
VIF

,feature,VIF
0,mph,3.066311
1,n_unicorns,3.426534
2,Idontknow,3.031613


所有的VIF都低於5以下，那我們就來重新計算一次回歸線

In [25]:
results = sm.OLS(y,X).fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                   fuel   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.612e+05
Date:                Mon, 04 Sep 2023   Prob (F-statistic):                        0.00
Time:                        00:50:47   Log-Likelihood:                         -313.99
No. Observations:                 200   AIC:                                      634.0
Df Residuals:                     197   BIC:                                      643.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

現在我們看到 mph 和fuel之間coefficient是1了。

多重共線性是普遍存在的，共線性的存在並不會對預測結果造成影響，如果模型僅用於預測，則只要擬合程度好，可不處理多重共線性問題，而在資料探勘中處理共線性可以增加解釋性並且使結果更穩定。
通常情況下，如果共線性情況不嚴重（VIF<5），不需要做特別的處理。

處理多重共線性可以手動剔除特徵(如上面演示)，在不便剔除特徵的情況下，可以使用**ridge regression**。